In [1]:
import os

%pwd # Print current directory

'/Users/wilsvenleong/Downloads/learning-materials/chicken-disease-classification/research'

In [2]:
os.chdir("../")  # Move up to root directory
%pwd

'/Users/wilsvenleong/Downloads/learning-materials/chicken-disease-classification'

In [3]:
import tensorflow as tf


model = tf.keras.models.load_model("artifacts/model_training/model.h5")

## 1. Update `params.yaml` and `config.yaml`

## 4. Update the Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model: Path
    training_data: Path
    all_params: dict
    image_size: list
    batch_size: int

## 5. Update the `ConfigurationManager` in `src/config/configuration.py`

In [5]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import (
    read_yaml,
    create_directories,
    save_json,
)


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir, "Chicken-fecal-images"
        )

        evaluation_config = EvaluationConfig(
            path_to_model=Path(self.config.model_training.trained_model_path),
            training_data=Path(training_data),
            all_params=self.params,
            image_size=self.params.IMAGE_SIZE,
            batch_size=self.params.BATCH_SIZE,
        )

        return evaluation_config

## 6. Update the Components

In [6]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.evaluation_config = config

    def _eval_valid_generator(self):
        data_generator_kwargs = {"rescale": 1.0 / 255, "validation_split": 0.3}
        data_flow_kwargs = {
            "target_size": self.evaluation_config.image_size[:-1],
            "batch_size": self.evaluation_config.batch_size,
            "interpolation": "bilinear",
        }

        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory=self.evaluation_config.training_data,
            subset="validation",
            shuffle=False,
            **data_flow_kwargs,
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        model = self.load_model(self.evaluation_config.path_to_model)
        self._eval_valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

## 7. Update the Pipeline

In [7]:
try:
    config_manager = ConfigurationManager()
    evaluation_config = config_manager.get_evaluation_config()
    evaluation = Evaluation(config=evaluation_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[ 2023-09-24 16:29:55,544 ] 33 common chicken-disease-logger -  INFO - Loaded YAML file successfully from: config/config.yaml
[ 2023-09-24 16:29:55,547 ] 33 common chicken-disease-logger -  INFO - Loaded YAML file successfully from: params.yaml
[ 2023-09-24 16:29:55,548 ] 53 common chicken-disease-logger -  INFO - Created directory at: artifacts
Found 116 images belonging to 2 classes.
8/8 [==============================] - 13s 2s/step - loss: 21.4065 - accuracy: 0.5000
[ 2023-09-24 16:30:08,555 ] 67 common chicken-disease-logger -  INFO - JSON file saved at: scores.json
